# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("wikipedia3/wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("wikipedia3/wikipedia_test3.csv", parse_dates=['date'])

In [3]:
train.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0


**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

### **k-fold validation**

In [4]:
def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=4))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(months=4)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=6))].index

In [5]:
train_dates = ['2016-01-01', '2016-02-01', '2016-03-01']

In [6]:
myCViterator = []
for i in train_dates:
    trainIndices, valIndices = create_validation(train, i)
    myCViterator.append( (trainIndices, valIndices) )

In [7]:
for x,y in myCViterator:
    print (min(x), min(y))

0 1429615
366265 1795880
708900 2150330


In [8]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [9]:
def prediction_15_days(train, test):
    max_date = max(train.date)
    start_date = max_date - relativedelta(days=15)
    train['Visits'].fillna(0, inplace=True)
    median_baseline = train.loc[train["date"] > start_date].groupby(['Page']).median().reset_index()
    median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)
    return test.merge(median_baseline, on="Page", how='left')

In [10]:
scores = []
for train_index, test_index in myCViterator:
    scores.append(pandas_smape(prediction_15_days(train.iloc[train_index], train.iloc[test_index])))

In [11]:
print(scores)

print('SMAPE on validation {}'.format(np.mean(scores)))

[39.352658930773224, 42.99083903078036, 41.01681045381974]
SMAPE on validation 41.12010280512444


In [12]:
print('SMAPE on test {}'.format(pandas_smape(prediction_15_days(train, test))))

SMAPE on test 40.007012702044264


### **holdout validation**

**time-based split**

In [13]:
train_date = '2016-06-03'
train_data = train.loc[train['date'] < train_date]
val_data = train.loc[train['date'] >= train_date]

print('Hold-out validation SMAPE {}'.format(pandas_smape(prediction_15_days(train_data, val_data))))

Hold-out validation SMAPE 45.72937213654275


In [14]:
print('Hold-out train SMAPE {}'.format(pandas_smape(prediction_15_days(train_data, train_data))))

Hold-out train SMAPE 45.65448880748438


## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [15]:
train = pd.read_csv('train_french.csv')
test = pd.read_csv('test_french.csv')

In [16]:
train.shape, test.shape

((200000, 29), (75968, 29))

In [17]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [18]:
import pandas_profiling as pp

In [19]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [20]:
pp.ProfileReport(test)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [21]:
from sklearn.linear_model import LinearRegression, LogisticRegression


In [22]:
reg = LinearRegression()

In [23]:
train['veranda'] = train['veranda'].map({'NON': 0, 'OUI': 1})
train['var9'] = train['var9'].map({'NON': 0, 'OUI': 1})

train['niv_urbanisation'] = train['niv_urbanisation'].map({'A': 1, 'B': 2, 'C': 3, 
                                                           'D': 4, 'E': 5, 'F': 6,
                                                           'G': 7, '~': 8})

train['sit_fam'] = train['sit_fam'].map({'Famille': 0, 'Indivividu': 1})

test['veranda'] = test['veranda'].map({'NON': 0, 'OUI': 1})
test['var9'] = test['var9'].map({'NON': 0, 'OUI': 1})

test['niv_urbanisation'] = test['niv_urbanisation'].map({'A': 1, 'B': 2, 'C': 3, 
                                                           'D': 4, 'E': 5, 'F': 6,
                                                           'G': 7, '~': 8})

test['sit_fam'] = test['sit_fam'].map({'Famille': 0, 'Indivividu': 1})

In [24]:
features = [
    'nb_pieces',
    'veranda',
    'biens_nomades',
    'societe',
    'domaine_activite',
    'niv_urbanisation',
    'age_societaire',
    'objets_precieux',
    'pres_contrat_auto',
    'formule',
    'sit_juridique',
    'surf_habitat',
    'patrimoine_mob',
    'var1', 
    'var2', 
    'var3', 
    'var4', 
    'var5',
    'var6',
    'var7',
    'var8',
    'var9', 
    'var10',
    'var11'
]

In [25]:
x_train, x_val, y_train, y_val = train_test_split(train[features], 
                                                   train['prime_tot_ttc'],
                                                   test_size=0.33, 
                                                   random_state=42)

In [26]:
def mape_score(y_true, y_pred):
    return np.sum(100. / y_true.shape[0] * np.abs((y_true - y_pred) / y_true))

In [27]:
reg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
print('Validation MAPE score {}'.format(mape_score(y_val, reg.predict(x_val))))

Validation MAPE score 18.507457022509705


In [29]:
print('Test MAPE score {}'.format(mape_score(test['prime_tot_ttc'], reg.predict(test[features]))))

Test MAPE score 18.79135731956065
